### ==========================================================================================
# PART 3: COVNET MODEL EVOLUTIONARY TRAINING
### ==========================================================================================

In [1]:
import os
import re
import random
import pandas as pd
import tensorflow as tf
from datetime import datetime
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from sklearn.metrics import confusion_matrix

Using TensorFlow backend.


In [2]:
# project folder
path_project = 'E:/DeepLearning/breast_cancer/IDC_regular_ps50_idx5/'

# set train, validate, and test folders
dir_train = os.path.join(path_project, '_train')
dir_valid = os.path.join(path_project, '_validate')

### Training Functions

In [3]:
def train_model(model_x, model_id, lr, epochs, epoch_steps, valid_steps, batch_size, image_size):
    model = model_x(lr=lr, image_size=image_size)
    
    print('Model:', model_x)
    print('Batch Size:', batch_size)
    print('Learning Rate:', lr)
    print('Epochs:', epochs)
    print('Steps per Epoch:', epoch_steps)
    print('Validation Steps:', valid_steps)
    print('Image Size:', image_size)
    print('_' * 65, '\n')
        
    print('Training Set:')    
    datagen_train = ImageDataGenerator(rescale=1./255)
    gen_train = datagen_train.flow_from_directory(dir_train, target_size=image_size, batch_size=batch_size, class_mode='binary')
    
    print('Validation Set:')    
    datagen_valid = ImageDataGenerator(rescale=1./255)
    gen_valid = datagen_valid.flow_from_directory(dir_valid, target_size=image_size, batch_size=batch_size, class_mode='binary')
    print('_' * 65, '\n')

    print('Model training started...\n')
    
    t0 = datetime.now()
    with tf.device('/gpu:0'):
        history = model.fit_generator(gen_train, steps_per_epoch=epoch_steps, epochs=epochs, validation_data=gen_valid, 
                                                                                                validation_steps=valid_steps)
    
    elpased_time = str(datetime.now() - t0).split('.')[0]    
    hours = elpased_time.split(':')[0]
    minutes = elpased_time.split(':')[1]
    seconds = elpased_time.split(':')[2]
    
    print('\nTraining process completed in:', int(hours), 'h', int(minutes), 'm', int(seconds), 's')            
    
    # save model
    models_folder = 'MODELS'    
    if not os.path.exists(os.path.join(path_project, models_folder)):
        os.makedirs(os.path.join(path_project, models_folder))
    filename = f"model_breast_cancer_{model_id}.h5"
    model.save(os.path.join(path_project, models_folder, filename))
    print('Model saved as: "', models_folder, '/', filename, '"')
    
    print('#' * 120, '\n')
    
    return (history, elpased_time, model)


def train_generation(model, image_size, gen):
    # read generations file
    df_generations = pd.read_csv(os.path.join(path_project, 'generations.csv')).fillna('')    

    for row in df_generations.itertuples():
        # process only untrained rows
        if row.train_loss == '' and row.val_acc == '':
            try:
                print('Processing Model', row.Index + 1, 'of', len(df_generations))
                logs = train_model(model, 
                                   model_id = row.Index,
                                   lr = row.learning_rate, 
                                   epochs = row.epochs, 
                                   epoch_steps = row.steps_per_epoch, 
                                   valid_steps = row.validation_steps, 
                                   batch_size = row.batch_size, 
                                   image_size = image_size)

                df_generations.loc[row.Index, 'train_loss'] = logs[0].history['loss'][-1]
                df_generations.loc[row.Index, 'train_acc'] = logs[0].history['acc'][-1]
                df_generations.loc[row.Index, 'val_loss'] = logs[0].history['val_loss'][-1]
                df_generations.loc[row.Index, 'val_acc'] = logs[0].history['val_acc'][-1]
                df_generations.loc[row.Index, 'time'] = logs[1]
                df_generations.loc[row.Index, 'gen'] = gen

                # save results to csv after each iteration
                df_generations.to_csv(os.path.join(path_project, 'generations.csv'), index=None)    
            except:
                pass
    
    return df_generations             

### Define Model Architecture

In [4]:
def model_7(lr, image_size):
    model = Sequential()
    
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)))
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Flatten())
    model.add(Dropout(0.5))
    
    model.add(Dense(512, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=lr), metrics=['acc'])        
    model.summary()    
    return model

### Define Genetic Algorithms

In [9]:
colnames = ['batch_size', 'steps_per_epoch', 'validation_steps', 'epochs', 'learning_rate',  
                                                    'train_loss', 'train_acc', 'val_loss', 'val_acc', 'time', 'gen', 'alive']

def first_generation(size):    
    df_out = pd.DataFrame()
    
    for i in range(size):
    
        # read generations file if it exists, or create new if it doesn't
        if os.path.exists(os.path.join(path_project, 'generations.csv')):
            df_generations = pd.read_csv(os.path.join(path_project, 'generations.csv'))
        else:
            df_generations = pd.DataFrame(columns=colnames)

        # create individual
        df_new = pd.DataFrame([[random.choice(range_batch_size),
                                random.choice(range_epoch_steps),
                                random.choice(range_valid_steps),
                                random.choice(range_epochs),
                                1/10**random.choice(range_lr),
                                '', '', '', '', '', '', '', '', '1', True]], columns=colnames)    
        df_temp = df_generations.append(df_new)

        # regenerate individual if already exists in df_generations
        while len(df_temp[df_temp.duplicated(keep=False)]) > 0:  
            df_generations = df_generations.drop_duplicates()
            df_new = pd.DataFrame([[random.choice(range_batch_size),
                                    random.choice(range_epoch_steps),
                                    random.choice(range_valid_steps),
                                    random.choice(range_epochs),
                                    1/10**random.choice(range_lr),
                                    '', '', '', '', '', '', '', '', '1', True]], columns=colnames)        
            df_temp = df_generations.append(df_new)

        df_out = df_out.append([df_new]).reset_index(drop=True)
            
        # save new parent to generations file
        df_temp.to_csv(os.path.join(path_project, 'generations.csv'), index=None)
            
    return df_out


def select_fittest(top_percent):
    # read generations file
    df_generations = pd.read_csv(os.path.join(path_project, 'generations.csv'))
    df_generations = df_generations.fillna('')

    # select a pool of only alive individuals
    df_alive = df_generations[df_generations.alive]

    # extinguish non-fit algorithms based on validation accuracy
    non_survivors = df_alive.nsmallest(round(len(df_alive) * (1 - top_percent / 100)), 'val_acc').index.tolist()    
    df_generations.loc[non_survivors, 'alive'] = False

    # save file
    df_generations.to_csv(os.path.join(path_project, 'generations.csv'), index=None)
    
    return df_generations[df_generations.alive]


def create_parents():    
    # read generations file
    df_generations = pd.read_csv(os.path.join(path_project, 'generations.csv'))
    
    # select a pool of only alive individuals
    df_pool = df_generations[df_generations.alive]
    
    # select parents at random from pool for as long as pairs can be matched 
    matched_parents = []
    while len(df_pool) > 1:    
        df_parents = df_pool.sample(2)
        new_parents = tuple(df_parents.index)
        matched_parents.append(new_parents)

        # remove new parents from pool        
        df_pool = df_pool.drop([new_parents[0], new_parents[1]], axis=0)

    return matched_parents


def create_child(df_parents, generation, mutate=False):
    df_generations = pd.read_csv(os.path.join(path_project, 'generations.csv'))
            
    mutate_choice = random.choice(['batch_size', 'steps_epoch', 'valid_steps', 'epochs', 'learning_rate'])  

    if mutate and mutate_choice == 'batch_size':
        batch_size = random.choice(range_batch_size)
    else:
        batch_size = int(df_parents.batch_size.sample())

    if mutate and mutate_choice == 'steps_epoch':
        steps_per_epoch = random.choice(range_epoch_steps)
    else:
        steps_per_epoch = int(df_parents.steps_per_epoch.sample())

    if mutate and mutate_choice == 'valid_steps':
        validation_steps = random.choice(range_valid_steps)
    else:
        validation_steps = int(df_parents.validation_steps.sample())

    if mutate and mutate_choice == 'epochs':
        epochs = random.choice(range_epochs)
    else:
        epochs = int(df_parents.epochs.sample())

    if mutate and mutate_choice == 'learning_rate':
        learning_rate = 1/10**random.choice(range_lr)
    else:
        learning_rate = float(df_parents.learning_rate.sample())            
        
    df_child = pd.DataFrame([[batch_size, steps_per_epoch, validation_steps, epochs, learning_rate,
                                                      '', '', '', '', '', generation, True]], columns=colnames)    
    df_temp = df_generations.append(df_child)
    
    # regenerate child if already exists in previous generations
    while len(df_temp[df_temp.duplicated(keep=False)]) > 0:  
        df_generations = df_generations.drop_duplicates()
        df_child = pd.DataFrame([[batch_size, steps_per_epoch, validation_steps, epochs, learning_rate,
                                      '', '', '', '', '', generation, True]], columns=colnames)    
        df_temp = df_generations.append(df_child)
        
    # save new child to generations file
    df_temp.to_csv(os.path.join(path_project, 'generations.csv'), index=None)
        
    return df_child

### Set Up Hyperparameter Combinations

In [6]:
range_epochs = range(10, 101, 10)
range_epoch_steps = range(10, 201, 10)
range_valid_steps = range(10, 101, 10)
range_batch_size = range(20, 201, 20)
range_lr = range(1, 6)

## 1st GENERATION ------------------------------------------------------------------

In [7]:
# Create first generation
first_generation(1000).tail()

,batch_size,steps_per_epoch,validation_steps,epochs,learning_rate,train_loss,train_acc,val_loss,val_acc,time,gen,alive
995,200,170,90,50,0.001,,,,,,1,True
996,160,140,10,80,0.001,,,,,,1,True
997,160,170,100,80,0.100,,,,,,1,True
998,180,190,100,70,0.001,,,,,,1,True
999,140,40,10,50,0.001,,,,,,1,True


In [8]:
# Train and validate
# see training logs folder for output
train_generation(model=model_7, image_size=(50,50), gen=1)

NOTE: See detailed output in "Training Logs" folder


In [9]:
# Keep top performing algorithms
df_fittest = select_fittest(top_percent=10)
df_fittest.nlargest(10, 'val_acc')

,batch_size,steps_per_epoch,validation_steps,epochs,learning_rate,train_loss,train_acc,val_loss,val_acc,time,gen,alive
653,100,190,10,90,0.0001,0.317655,0.865789,0.292571,0.894000,0:11:41,1,True
994,140,180,50,50,0.0010,0.320372,0.864048,0.315853,0.874857,0:10:14,1,True
693,140,160,10,80,0.0010,0.315224,0.867411,0.319296,0.874613,0:13:23,1,True
605,60,200,30,80,0.0010,0.326009,0.861417,0.316879,0.873889,0:19:31,1,True
760,200,190,40,100,0.0001,0.298031,0.874026,0.304660,0.873125,0:39:18,1,True
963,140,200,90,90,0.0001,0.312412,0.867250,0.306135,0.872778,0:34:51,1,True
548,80,200,30,100,0.0010,0.313608,0.866063,0.324402,0.872083,0:21:46,1,True
996,160,140,10,80,0.0010,0.309834,0.872634,0.339324,0.871875,0:12:31,1,True
276,100,170,50,80,0.0001,0.335177,0.857294,0.311656,0.871600,0:24:30,1,True
215,180,130,90,100,0.0010,0.304454,0.868462,0.312375,0.871169,0:37:29,1,True


## 2nd GENERATION ------------------------------------------------------------------

In [10]:
parents = create_parents()
parents

[(276, 936),
 (258, 46),
 (170, 839),
 (968, 246),
 (242, 367),
 (521, 713),
 (853, 302),
 (630, 557),
 (114, 714),
 (605, 980),
 (653, 996),
 (626, 363),
 (256, 591),
 (961, 15),
 (652, 480),
 (215, 760),
 (574, 795),
 (90, 454),
 (93, 88),
 (10, 326),
 (553, 142),
 (418, 266),
 (430, 924),
 (444, 693),
 (781, 496),
 (688, 813),
 (994, 888),
 (982, 892),
 (451, 132),
 (638, 167),
 (95, 269),
 (904, 442),
 (216, 685),
 (355, 158),
 (76, 706),
 (358, 229),
 (801, 548),
 (96, 768),
 (998, 960),
 (716, 632),
 (372, 977),
 (137, 724),
 (832, 176),
 (887, 775),
 (787, 174),
 (275, 938),
 (995, 814),
 (202, 953),
 (437, 163),
 (963, 923)]

In [13]:
# read generations file
df_generations = pd.read_csv(os.path.join(path_project, 'generations.csv')).fillna('')
    
for pair in parents:
    df_parents = df_generations.iloc[[pair[0], pair[1]]]
    
    # make children
    df_child_1 = create_child(df_parents, generation=2)
    df_child_2 = create_child(df_parents, generation=2)
    df_child_3 = create_child(df_parents, generation=2)
    df_child_4 = create_child(df_parents, generation=2)
    df_child_5 = create_child(df_parents, generation=2, mutate=True)

    df_children = df_child_1.append([df_child_2, df_child_3, df_child_4, df_child_5], ignore_index=True)

In [14]:
# example of parents
df_parents

,batch_size,steps_per_epoch,validation_steps,epochs,learning_rate,train_loss,train_acc,val_loss,val_acc,time,gen,alive
963,140,200,90,90,0.0001,0.312412,0.867250,0.306135,0.872778,0:34:51,1,True
923,80,110,50,90,0.0001,0.334619,0.858523,0.333084,0.854750,0:07:29,1,True


In [15]:
# example of children
df_children

,batch_size,steps_per_epoch,validation_steps,epochs,learning_rate,train_loss,train_acc,val_loss,val_acc,time,gen,alive
0,80,200,90,90,0.0001,,,,,,2,True
1,140,200,50,90,0.0001,,,,,,2,True
2,80,110,50,90,0.0001,,,,,,2,True
3,80,110,50,90,0.0001,,,,,,2,True
4,140,200,90,90,0.1000,,,,,,2,True


In [9]:
# Train and validate
# see training logs folder for output
train_generation(model=model_7, image_size=(50,50), gen=2)

NOTE: See detailed output in "Training Logs" folder


In [10]:
# Keep top performing algorithms
df_fittest = select_fittest(top_percent=10)
df_fittest.nlargest(10, 'val_acc')

,batch_size,steps_per_epoch,validation_steps,epochs,learning_rate,train_loss,train_acc,val_loss,val_acc,time,gen,alive
653,100,190,10,90,0.0001,0.317655,0.865789,0.292571,0.894000,0:11:41,1,True
1178,80,190,30,50,0.0010,0.333464,0.858750,0.292957,0.884583,0:05:13,2,True
1129,200,200,20,70,0.0001,0.311461,0.867375,0.295427,0.881832,0:29:50,2,True
1107,80,160,10,80,0.0010,0.321133,0.866250,0.336637,0.881250,0:10:42,2,True
1038,100,200,10,90,0.0010,0.306632,0.870800,0.317246,0.881000,0:14:31,2,True
1039,100,190,10,50,0.0001,0.344512,0.852526,0.317177,0.878000,0:07:40,2,True
1029,200,150,20,90,0.0010,0.294208,0.877200,0.308905,0.877500,0:21:22,2,True
1106,140,200,10,80,0.0001,0.316372,0.865000,0.288847,0.876935,0:21:26,2,True
1047,160,140,10,80,0.0010,0.301332,0.874420,0.303735,0.876875,0:14:22,2,True
1125,200,120,90,100,0.0010,0.299698,0.874125,0.298709,0.876500,0:40:10,2,True


## 3rd GENERATION ------------------------------------------------------------------

In [11]:
parents = create_parents()
parents

[(1150, 1074),
 (994, 693),
 (1083, 653),
 (1178, 1047),
 (996, 1067),
 (1068, 605),
 (1106, 1016),
 (1039, 1119),
 (1142, 963),
 (760, 548),
 (1125, 1107),
 (1032, 1134),
 (1029, 1103),
 (1176, 1171),
 (1038, 1129),
 (276, 1112)]

In [12]:
# read generations file
df_generations = pd.read_csv(os.path.join(path_project, 'generations.csv')).fillna('')
    
for pair in parents:
    df_parents = df_generations.iloc[[pair[0], pair[1]]]
    
    # make children
    df_child_1 = create_child(df_parents, generation=3)
    df_child_2 = create_child(df_parents, generation=3)
    df_child_3 = create_child(df_parents, generation=3)
    df_child_4 = create_child(df_parents, generation=3)
    df_child_5 = create_child(df_parents, generation=3, mutate=True)

    df_children = df_child_1.append([df_child_2, df_child_3, df_child_4, df_child_5], ignore_index=True)

In [13]:
df_parents

,batch_size,steps_per_epoch,validation_steps,epochs,learning_rate,train_loss,train_acc,val_loss,val_acc,time,gen,alive
276,100,170,50,80,0.0001,0.335177,0.857294,0.311656,0.871600,0:24:30,1,True
1112,180,160,60,100,0.0010,0.293139,0.878368,0.306576,0.873426,0:34:54,2,True


In [14]:
df_children

,batch_size,steps_per_epoch,validation_steps,epochs,learning_rate,train_loss,train_acc,val_loss,val_acc,time,gen,alive
0,180,170,50,80,0.0001,,,,,,3,True
1,180,170,60,80,0.0010,,,,,,3,True
2,180,160,50,100,0.0010,,,,,,3,True
3,100,160,60,80,0.0010,,,,,,3,True
4,180,10,50,100,0.0010,,,,,,3,True


In [15]:
# Train and validate
# see training logs folder for output
train_generation(model=model_7, image_size=(50,50), gen=3)

NOTE: See detailed output in "Training Logs" folder


In [16]:
# Keep top performing algorithms
df_fittest = select_fittest(top_percent=10)
df_fittest.nlargest(10, 'val_acc')

,batch_size,steps_per_epoch,validation_steps,epochs,learning_rate,train_loss,train_acc,val_loss,val_acc,time,gen,alive
653,100,190,10,90,0.0001,0.317655,0.865789,0.292571,0.894000,0:11:41,1,True
1240,160,140,10,80,0.0010,0.309508,0.870714,0.298903,0.886250,0:12:35,3,True
1178,80,190,30,50,0.0010,0.333464,0.858750,0.292957,0.884583,0:05:13,2,True
1278,200,130,10,90,0.0010,0.302942,0.873192,0.296192,0.882000,0:17:44,3,True
1129,200,200,20,70,0.0001,0.311461,0.867375,0.295427,0.881832,0:29:50,2,True
1107,80,160,10,80,0.0010,0.321133,0.866250,0.336637,0.881250,0:10:42,2,True
1038,100,200,10,90,0.0010,0.306632,0.870800,0.317246,0.881000,0:14:31,2,True
1250,200,190,30,100,0.0010,0.284835,0.881026,0.298489,0.878500,0:28:42,3,True
1267,200,150,20,60,0.0010,0.310335,0.870633,0.298475,0.878250,0:13:40,3,True
1039,100,190,10,50,0.0001,0.344512,0.852526,0.317177,0.878000,0:07:40,2,True


## 4th GENERATION ------------------------------------------------------------------

In [17]:
parents = create_parents()
parents

[(1039, 1278), (1250, 1129), (1107, 1267), (653, 1029), (1178, 1240)]

In [18]:
# read generations file
df_generations = pd.read_csv(os.path.join(path_project, 'generations.csv')).fillna('')
    
for pair in parents:
    df_parents = df_generations.iloc[[pair[0], pair[1]]]
    
    # make children
    df_child_1 = create_child(df_parents, generation=4)
    df_child_2 = create_child(df_parents, generation=4)
    df_child_3 = create_child(df_parents, generation=4)
    df_child_4 = create_child(df_parents, generation=4)
    df_child_5 = create_child(df_parents, generation=4, mutate=True)

    df_children = df_child_1.append([df_child_2, df_child_3, df_child_4, df_child_5], ignore_index=True)

In [19]:
df_parents

,batch_size,steps_per_epoch,validation_steps,epochs,learning_rate,train_loss,train_acc,val_loss,val_acc,time,gen,alive
1178,80,190,30,50,0.001,0.333464,0.858750,0.292957,0.884583,0:05:13,2,True
1240,160,140,10,80,0.001,0.309508,0.870714,0.298903,0.886250,0:12:35,3,True


In [20]:
df_children

,batch_size,steps_per_epoch,validation_steps,epochs,learning_rate,train_loss,train_acc,val_loss,val_acc,time,gen,alive
0,160,190,30,80,0.001,,,,,,4,True
1,80,140,10,80,0.001,,,,,,4,True
2,80,190,30,80,0.001,,,,,,4,True
3,160,190,10,50,0.001,,,,,,4,True
4,80,120,30,80,0.001,,,,,,4,True


In [21]:
# Train and validate
# see training logs folder for output
train_generation(model=model_7, image_size=(50,50), gen=4)

NOTE: See detailed output in "Training Logs" folder


In [22]:
# Get number of current live algorithms
df_generations = pd.read_csv(os.path.join(path_project, 'generations.csv')).fillna('')
len(df_generations[df_generations.alive])

35

In [23]:
# Keep top performing algorithms
df_fittest = select_fittest(top_percent=50)
df_fittest.nlargest(10, 'val_acc')

,batch_size,steps_per_epoch,validation_steps,epochs,learning_rate,train_loss,train_acc,val_loss,val_acc,time,gen,alive
653,100,190,10,90,0.0001,0.317655,0.865789,0.292571,0.894000,0:11:41,1,True
1240,160,140,10,80,0.0010,0.309508,0.870714,0.298903,0.886250,0:12:35,3,True
1178,80,190,30,50,0.0010,0.333464,0.858750,0.292957,0.884583,0:05:13,2,True
1278,200,130,10,90,0.0010,0.302942,0.873192,0.296192,0.882000,0:17:44,3,True
1129,200,200,20,70,0.0001,0.311461,0.867375,0.295427,0.881832,0:29:50,2,True
1107,80,160,10,80,0.0010,0.321133,0.866250,0.336637,0.881250,0:10:42,2,True
1038,100,200,10,90,0.0010,0.306632,0.870800,0.317246,0.881000,0:14:31,2,True
1250,200,190,30,100,0.0010,0.284835,0.881026,0.298489,0.878500,0:28:42,3,True
1267,200,150,20,60,0.0010,0.310335,0.870633,0.298475,0.878250,0:13:40,3,True
1039,100,190,10,50,0.0001,0.344512,0.852526,0.317177,0.878000,0:07:40,2,True


### Analyze Generations

In [24]:
# read generations file
df_generations = pd.read_csv(os.path.join(path_project, 'generations.csv')).fillna('')

# check performance of top performing algorithms (dead and alive)
df_generations.sort_values('val_acc', ascending=False).head(30)

,batch_size,steps_per_epoch,validation_steps,epochs,learning_rate,train_loss,train_acc,val_loss,val_acc,time,gen,alive
653,100,190,10,90,0.0001,0.317655,0.865789,0.292571,0.894000,0:11:41,1,True
1240,160,140,10,80,0.0010,0.309508,0.870714,0.298903,0.886250,0:12:35,3,True
1178,80,190,30,50,0.0010,0.333464,0.858750,0.292957,0.884583,0:05:13,2,True
1278,200,130,10,90,0.0010,0.302942,0.873192,0.296192,0.882000,0:17:44,3,True
1129,200,200,20,70,0.0001,0.311461,0.867375,0.295427,0.881832,0:29:50,2,True
1107,80,160,10,80,0.0010,0.321133,0.866250,0.336637,0.881250,0:10:42,2,True
1038,100,200,10,90,0.0010,0.306632,0.870800,0.317246,0.881000,0:14:31,2,True
1250,200,190,30,100,0.0010,0.284835,0.881026,0.298489,0.878500,0:28:42,3,True
1267,200,150,20,60,0.0010,0.310335,0.870633,0.298475,0.878250,0:13:40,3,True
1039,100,190,10,50,0.0001,0.344512,0.852526,0.317177,0.878000,0:07:40,2,True


### Total Training Time of All Generations

In [27]:
df_generations = pd.read_csv(os.path.join(path_project, 'generations.csv')).fillna('')

times = df_generations.time.tolist()
times = list(map(lambda x: re.sub(r'^0:', '', x), times))

minutes = sum(list(map(lambda x: int(x.split(':')[0]), times))) + sum(list(map(lambda x: int(x.split(':')[1]), times))) / 60
hours = minutes/60
print(round(hours, 2), 'hours')

237.54 hours
